In [ ]:
import http.client

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-apisports-key': "261d42af5fbae54096924d7e009a10b1"
}

conn.request("GET", "/leagues", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


Rquest

In [ ]:
url = "https://v3.football.api-sports.io/leagues"

payload={}
headers = {
  'x-apisports-key': '261d42af5fbae54096924d7e009a10b1',
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)